<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/nwds_k_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# font

In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)

<IPython.core.display.Javascript object>

# Benchmark

In [ ]:
!pip install catboost

<IPython.core.display.Javascript object>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost as cb

project_dir = 'drive/My Drive/nwds-k-benchmark'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
os.listdir(project_dir)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

['test.csv', 'sample_solution.csv', 'train.csv']

In [ ]:
train = pd.read_csv(f'{project_dir}/train.csv')
test = pd.read_csv(f'{project_dir}/test.csv')
sample_solution = pd.read_csv(f'{project_dir}/sample_solution.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train['is_lhb'] = 0
train.loc[train['stand']=='L', 'is_lhb'] = 1

train['is_lhp'] = 0
train.loc[train['p_throws']=='L', 'is_lhp'] = 1

train['is_bot'] = 0
train.loc[train['inning_topbot']=='Bot', 'is_bot'] = 1

train['bat_speed'] = train['bat_speed'].fillna(-1)
train['swing_length'] = train['swing_length'].fillna(-1)

train['pitch_type_code'] = train['pitch_type'].astype('category').cat.codes
pt_map = train.loc[:, ['pitch_type','pitch_type_code']].drop_duplicates().set_index('pitch_type').to_dict()['pitch_type_code']

test['is_lhb'] = 0
test.loc[test['stand']=='L', 'is_lhb'] = 1

test['is_lhp'] = 0
test.loc[test['p_throws']=='L', 'is_lhp'] = 1

test['is_bot'] = 0
test.loc[test['inning_topbot']=='Bot', 'is_bot'] = 1

test['bat_speed'] = test['bat_speed'].fillna(-1)
test['swing_length'] = test['swing_length'].fillna(-1)

test['pitch_type_code'] = test['pitch_type'].map(pt_map)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
feats = [
    'on_3b', 'on_2b', 'on_1b',
    'inning', 'outs_when_up', 'balls', 'strikes',
    'n_thruorder_pitcher', 'sz_top', 'sz_bot',
    'pfx_x', 'pfx_z', 'arm_angle', 'release_speed',
    'release_pos_x', 'release_extension', 'release_pos_z',
    'release_spin_rate', 'spin_axis', 'bat_speed',
    'swing_length', 'is_lhb', 'is_lhp', 'is_bot',
    'pitch_type_code'
]
target = 'is_strike'

model = cb.CatBoostClassifier(verbose=False)
model.fit(train.loc[:, feats], train[target])
output = pd.DataFrame(model.predict_proba(test.loc[:, feats]), index=test.index)
test['k'] = output[1]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test.loc[:, ['index', 'k']].to_csv('nwds-benchmark.csv', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>